<a href="https://colab.research.google.com/github/tpmayys/demo1/blob/main/NER_WangchanBerta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Install Library

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# dev version
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp -q
! pip install epitran -q
! pip install sklearn_crfsuite -q
!pip install evaluate -q
!pip install seqeval -q

     |████████████████████████████████| 9.6 MB 5.0 MB/s 
     |████████████████████████████████| 164 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 15.7 MB/s 
     |████████████████████████████████| 73 kB 677 kB/s 
     |████████████████████████████████| 965 kB 5.1 MB/s 


In [ ]:
import pythainlp
pythainlp.__version__

'3.1.1'

####Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
traffy_df = pd.read_csv("/content/drive/MyDrive/DSI444/2000 - Sheet1 (5).csv")

In [ ]:
traffy_df = traffy_df.dropna()

In [ ]:
traffy_df = traffy_df.reset_index(drop=True)

In [ ]:
traffy_df

,ticket_id,type,organization,comment,coords,photo,address,district,subdistrict,province,timestamp,state
0,2022-4EUFUP,ป้าย,กรุงเทพมหานคร,ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง,"100.54987,13.74826",https://storage.googleapis.com/traffy_public_b...,ทางพิเศษเฉลิมมหานคร แขวง มักกะสัน เขตราชเทวี ก...,ปทุมวัน,ลุมพินี,กรุงเทพมหานคร,2022-10-20 06:45:42.413105+00,รอรับเรื่อง
1,2022-DYAPUZ,สัตว์จรจัด,กรุงเทพมหานคร,น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...,"100.69905,13.83208",https://storage.googleapis.com/traffy_public_b...,428/68 The regent street แขวง บางชัน เขตคลองสา...,คลองสามวา,บางชัน,กรุงเทพมหานคร,2022-10-20 06:44:41.296435+00,รอรับเรื่อง
2,2022-A4ELMK,ท่อระบายน้ำ,กรุงเทพมหานคร,ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...,"100.61712,13.87396",https://storage.googleapis.com/traffy_public_b...,9 ชั้น 2 ซอย รามอินทรา 5 แยก 15 แขวง อนุสาวรีย...,บางเขน,อนุสาวรีย์,กรุงเทพมหานคร,2022-10-20 06:36:20.402378+00,กำลังดำเนินการ
3,2022-AMXD6N,"ความสะอาด,สอบถาม,สัตว์จรจัด",กรุงเทพมหานคร,สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...,"100.48619,13.71561",https://storage.googleapis.com/traffy_public_b...,965 ถนน สมเด็จพระเจ้าตากสิน แขวง บุคคโล เขตธนบ...,ธนบุรี,บุคคโล,กรุงเทพมหานคร,2022-10-20 06:34:28.448729+00,กำลังดำเนินการ
4,2022-GAZZ2B,ท่อระบายน้ำ,กรุงเทพมหานคร,ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15,"100.61763,13.87367",https://storage.googleapis.com/traffy_public_b...,เลขที่ 2 ซอย รามอินทรา 5 แยก 15 แขวง อนุสาวรีย...,บางเขน,อนุสาวรีย์,กรุงเทพมหานคร,2022-10-20 06:34:00.039707+00,กำลังดำเนินการ
...,...,...,...,...,...,...,...,...,...,...,...,...
1611,2022-L72R66,"ถนน,ความปลอดภัย",กรุงเทพมหานคร,ไฟจราจรเสีย ถนนวุ่นวายมาก,"100.45047,13.62656",https://storage.googleapis.com/traffy_public_b...,540 ซ. อนามัยงามเจริญ แขวง ท่าข้าม เขตบางขุนเท...,บางขุนเทียน,ท่าข้าม,กรุงเทพมหานคร,2022-10-15 06:27:54.343198+00,รอรับเรื่อง
1612,2022-ATYR6D,ทางเท้า,กรุงเทพมหานคร,ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...,"100.58422,13.80141",https://storage.googleapis.com/traffy_public_b...,26 ซ. ลาดพร้าว 41 แขวง สามเสนนอก เขตห้วยขวาง ก...,ห้วยขวาง,สามเสนนอก,กรุงเทพมหานคร,2022-10-15 06:22:15.182445+00,กำลังดำเนินการ
1613,2022-FY468F,ความสะอาด,กรุงเทพมหานคร,ถังขยะล้นและพัง ขยะปลิ้วออกมา,"100.62854,13.74393",https://storage.googleapis.com/traffy_public_b...,1008 ถนน พระราม 9 แขวง สวนหลวง แขวงสวนหลวง กรุ...,สวนหลวง,สวนหลวง,กรุงเทพมหานคร,2022-10-15 06:15:02.484086+00,เสร็จสิ้น
1614,2022-MQZY3E,ต้นไม้,กรุงเทพมหานคร,ดืนสุดตัวกลัวต้นไม้ล้มค่ะ\n(หนูขุดหลายรู),"100.52211,13.72391",https://storage.googleapis.com/traffy_public_b...,833 ถนน สีลม แขวง สีลม เขตบางรัก กรุงเทพมหานคร...,บางรัก,สีลม,กรุงเทพมหานคร,2022-10-15 06:14:56.988404+00,เสร็จสิ้น


In [ ]:
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U00002500-\U00002BEF"  # chinese ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200b"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # dingbats
                           u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_symbol_someWord(text):
    text = re.sub(r'\n|[\n]+', '', text)
    return text

In [ ]:
traffy_df['comment'] = traffy_df['comment'].apply(lambda x: remove_emoji(x))
traffy_df['comment'] = traffy_df['comment'].apply(lambda x: remove_symbol_someWord(x))

In [ ]:
traffy_df['comment'].replace('กทม', 'กรุงเทพมหานคร')

0                        ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง
1       น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...
2       ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...
3       สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...
4        ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15
                              ...                        
1611                            ไฟจราจรเสีย ถนนวุ่นวายมาก
1612    ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...
1613                        ถังขยะล้นและพัง ขยะปลิ้วออกมา
1614              ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู)
1615    ทางเดินริมคลองไม่ต่อเนื่อง โดนสะพานข้ามขวางกั้...
Name: comment, Length: 1616, dtype: object

In [ ]:
traffy_df['comment'].replace('กทม.', 'กรุงเทพมหานคร')

0                        ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง
1       น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...
2       ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...
3       สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...
4        ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15
                              ...                        
1611                            ไฟจราจรเสีย ถนนวุ่นวายมาก
1612    ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...
1613                        ถังขยะล้นและพัง ขยะปลิ้วออกมา
1614              ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู)
1615    ทางเดินริมคลองไม่ต่อเนื่อง โดนสะพานข้ามขวางกั้...
Name: comment, Length: 1616, dtype: object

In [ ]:
traffy_df['comment'].replace('กรุงเทพ', 'กรุงเทพมหานคร')

0                        ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง
1       น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...
2       ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...
3       สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...
4        ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15
                              ...                        
1611                            ไฟจราจรเสีย ถนนวุ่นวายมาก
1612    ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...
1613                        ถังขยะล้นและพัง ขยะปลิ้วออกมา
1614              ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู)
1615    ทางเดินริมคลองไม่ต่อเนื่อง โดนสะพานข้ามขวางกั้...
Name: comment, Length: 1616, dtype: object

In [ ]:
traffy_df['comment'].replace('สนง.', 'สำนักงาน')

0                        ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง
1       น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...
2       ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...
3       สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...
4        ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15
                              ...                        
1611                            ไฟจราจรเสีย ถนนวุ่นวายมาก
1612    ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...
1613                        ถังขยะล้นและพัง ขยะปลิ้วออกมา
1614              ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู)
1615    ทางเดินริมคลองไม่ต่อเนื่อง โดนสะพานข้ามขวางกั้...
Name: comment, Length: 1616, dtype: object

In [ ]:
df = traffy_df['comment']
df

0                        ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง
1       น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธิ...
2       ฝาท่อชำรุดบริเวณรามอินทรา 5 แยก 15 ลานจอดรถตรง...
3       สอบถามเรื่องการเก็บขยะเขตธนบุรี ซอยสมเด็จพระเจ...
4        ฝาท่อเมนชำรุดบริเวณร้านของเก่ารามอินทรา 5 แยก 15
                              ...                        
1611                            ไฟจราจรเสีย ถนนวุ่นวายมาก
1612    ฟุตบาทมีความขรุขระ ไม่เรียบเสมอกัน ส่งผลต่อการ...
1613                        ถังขยะล้นและพัง ขยะปลิ้วออกมา
1614              ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู)
1615    ทางเดินริมคลองไม่ต่อเนื่อง โดนสะพานข้ามขวางกั้...
Name: comment, Length: 1616, dtype: object

In [ ]:
df.shape

(1616,)

###NER

In [ ]:
!pip install transformers sentencepiece  # install for pythainlp 3 wangchanberta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 1.3 MB 66.5 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 182 kB 72.5 MB/s 


In [ ]:
from pythainlp.wangchanberta import ThaiNameTagger # pythainlp 3 wangchanberta
ner = ThaiNameTagger()
t = ThaiNameTagger(dataset_name="thainer")

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pythainlp/wangchanberta/postag.py:30: UserWarning: 
            LST20 corpus are free for research and open source only.

            If you want to use in Commercial use, please contract NECTEC.

            https://www.facebook.com/dancearmy/posts/10157641945708284
            
  """)


Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  "`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to"


Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  "`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to"


In [ ]:
pyner1 = []
for data in df:
  ne = ner.get_ner(data)
  pyner1.append(ne)

In [ ]:
pyner1

[[('ป้ายเตือนบนทางด่วนหัก  เสี่ยงร่วง', 'O')],
 [('น้องหมาถูกรถชนขาหักครับ  ตอนนี้แจ้งไปทางมูลนิธิต่างๆแล้วยังไม่มีที่ไหนตอบรับครับ',
   'O')],
 [('ฝาท่อชํารุด', 'O'),
  ('บริเวณ', 'B-LOCATION'),
  ('รามอินทรา', 'I-LOCATION'),
  (' ', 'O'),
  ('5', 'B-LOCATION'),
  (' ', 'O'),
  ('', 'B-LOCATION'),
  ('แยก', 'I-LOCATION'),
  (' ', 'O'),
  ('15', 'B-LOCATION'),
  ('  ', 'O'),
  ('ลานจอดรถ', 'B-LOCATION'),
  ('ตรงข้าม', 'I-LOCATION'),
  ('บริษัทเป็นต่อ', 'B-ORGANIZATION')],
 [('สอบถามเรื่องการเก็บขยะ', 'O'),
  ('เขต', 'B-LOCATION'),
  ('ธนบุรี', 'I-LOCATION'),
  (' ', 'O'),
  ('', 'B-LOCATION'),
  ('ซอยสมเด็จพระเจ้าตากสิน', 'I-LOCATION'),
  (' ', 'O'),
  ('18', 'B-LOCATION'),
  ('  ปกติจะมีเจ้าหน้าที่มาเก็บขยะตามหน้าบ้านที่วางไว้ทุกวัน  แต่ระยะหลังนี้บางวันเจ้าหน้าที่ก็ไม่มาเก็บ  ถ้าจะมีวันไหนที่จะไม่มาเก็บอยากให้มีการแจ้งเนื่องจากบางครั้งมีหมาแมวมาคุ้ยถุงขยะ',
   'O')],
 [('ฝาท่อเมนชํารุดบริเวณ', 'O'),
  ('ร้าน', 'B-LOCATION'),
  ('ของเก่า', 'I-LOCATION'),
  ('รามอินทรา  5', 'I-LOCATION

####Word Cloud

#####Prepare for Word Cloud

In [ ]:
df_list = []
for i in range(len(pyner1)):
    my_df = pd.DataFrame(pyner1[i], columns=["word", "tag"])
    df_list.append(my_df)
df_list

[                                word tag
 0  ป้ายเตือนบนทางด่วนหัก  เสี่ยงร่วง   O,
                                                 word tag
 0  น้องหมาถูกรถชนขาหักครับ  ตอนนี้แจ้งไปทางมูลนิธ...   O,
              word             tag
 0     ฝาท่อชํารุด               O
 1          บริเวณ      B-LOCATION
 2       รามอินทรา      I-LOCATION
 3                               O
 4               5      B-LOCATION
 5                               O
 6                      B-LOCATION
 7             แยก      I-LOCATION
 8                               O
 9              15      B-LOCATION
 10                              O
 11       ลานจอดรถ      B-LOCATION
 12        ตรงข้าม      I-LOCATION
 13  บริษัทเป็นต่อ  B-ORGANIZATION,
                                                 word         tag
 0                             สอบถามเรื่องการเก็บขยะ           O
 1                                                เขต  B-LOCATION
 2                                             ธนบุรี  I-LOCATION
 3      

In [ ]:
az = pd.concat(df_list, axis=0)

In [ ]:
az.reset_index(drop=True)

,word,tag
0,ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง,O
1,น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธ...,O
2,ฝาท่อชํารุด,O
3,บริเวณ,B-LOCATION
4,รามอินทรา,I-LOCATION
...,...,...
8939,สุข,I-LOCATION
8940,15,I-LOCATION
8941,ถังขยะล้นและพัง ขยะปลิ้วออกมา,O
8942,ดืนสุดตัวกลัวต้นไม้ล้มค่ะ(หนูขุดหลายรู),O


In [ ]:
az.to_csv("/content/drive/MyDrive/DSI444/Wangchan_prediction.csv", index=False)

#Evaluation

In [ ]:
import pandas as pd

ner_pred = pd.read_csv("/content/drive/MyDrive/DSI444/Wangchan_prediction.csv")
ner_pred = ner_pred[0:853].copy()
# ner_pred = ner_pred.drop(index=[23, 26, 33])
# ner_pred = ner_pred.reset_index(drop=True)
ner_pred

,word,tag
0,ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง,O
1,น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธ...,O
2,ฝาท่อชํารุด,O
3,บริเวณ,B-LOCATION
4,รามอินทรา,I-LOCATION
...,...,...
848,.ทั้งเสียชื่อ.ผู้ว่า,O
849,.,B-ORGANIZATION
850,กทม,I-ORGANIZATION
851,.,I-ORGANIZATION


In [ ]:
ner_label = pd.read_csv("/content/drive/MyDrive/DSI444/Wangchan_prediction_label - Sheet1.csv")
# ner_label = ner_label.drop(index=[23, 26, 33])
# ner_label = ner_label.reset_index(drop=True)
ner_label

,word,tag
0,ป้ายเตือนบนทางด่วนหัก เสี่ยงร่วง,O
1,น้องหมาถูกรถชนขาหักครับ ตอนนี้แจ้งไปทางมูลนิธ...,O
2,ฝาท่อชํารุด,O
3,บริเวณ,B-LOCATION
4,รามอินทรา,I-LOCATION
...,...,...
848,.ทั้งเสียชื่อ.ผู้ว่า,O
849,.,O
850,กทม,O
851,.,O


In [ ]:
import re

def replace_like_list(text_list):
  text_list = re.sub(r"\'", "", text_list)
  text_list = re.sub(r"\[", "", text_list)
  text_list = re.sub(r"\]", "", text_list)

  return text_list

In [ ]:
ner_pred["tag"] = ner_pred["tag"].apply(lambda x: replace_like_list(x))
ner_label["tag"] = ner_label["tag"].apply(lambda x: replace_like_list(x))

In [ ]:
ner_pred["tag"] = ner_pred["tag"].apply(lambda x: x.split(","))
ner_label["tag"] = ner_label["tag"].apply(lambda x: x.split(","))

In [ ]:
full_tag = []

for tag in ner_label["ner_tag"]:
  ner_tag = []
  for ner in tag:
    ner = ner.strip()
    ner_tag.append(ner)
    if " " in ner:
      print(tag)
  
  full_tag.append(ner_tag)

In [ ]:
ner_label["new_ner_tag"] = full_tag
ner_label

In [ ]:
import evaluate

seqeval = evaluate.load('seqeval')
predictions = ner_pred["tag"].values.tolist()
references = ner_label["tag"].values.tolist()
results = seqeval.compute(predictions=predictions, references=references)
results

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'DATE': {'precision': 0.975609756097561,
  'recall': 1.0,
  'f1': 0.9876543209876543,
  'number': 40},
 'LAW': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'LEN': {'precision': 0.42857142857142855,
  'recall': 0.75,
  'f1': 0.5454545454545454,
  'number': 4},
 'LOCATION': {'precision': 0.8354430379746836,
  'recall': 0.967032967032967,
  'f1': 0.8964346349745331,
  'number': 273},
 'ORGANIZATION': {'precision': 0.7209302325581395,
  'recall': 1.0,
  'f1': 0.8378378378378378,
  'number': 31},
 'PHONE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'TIME': {'precision': 0.8823529411764706,
  'recall': 1.0,
  'f1': 0.9375,
  'number': 45},
 'overall_precision': 0.832258064516129,
 'overall_recall': 0.9748110831234257,
 'overall_f1': 0.8979118329466358,
 'overall_accuracy': 0.8311840562719812}

####Prepare Data

In [ ]:
az.astype(str)

In [ ]:
BLocation = az.groupby(['tag']).get_group('B-LOCATION').reset_index(drop=True)
ILocation = az.groupby(['tag']).get_group('I-LOCATION').reset_index(drop=True)
Location = pd.concat([BLocation, ILocation], axis=0)
Location

In [ ]:
#เอาคำที่ไม่เกี่ยวข้องออก
Location.drop(Location.loc[Location['word'] == 'บ้านเลขที่่'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ถนน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'เขต'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ปากซอย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'เส้น'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'หน้า'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'แยก'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ซอย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สะพาน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ซ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'พระราม'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สะพานข้าม'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ถ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'บริเวณ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ป้าย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ใต้'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'แขวง'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'เลียบ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ขาเข้า'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'และ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'กลับรถ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ข้าม'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สวน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'โอ'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ไทย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'หมู่บ้าน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ถนนพระราม'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ร้าน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ในซอย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'แยก '].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ซอย	'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'คลอง'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'บาง'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สถานี'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == ' แยก'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ฝั่ง'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ตรงข้าม'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'กรุง'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'ตลาด'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'สี่แยก'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == ' สี่แยก '].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'บ้านเลขที่'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'บนถนน'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'เเยก'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == ' ซอย'].index, inplace=True)
Location.drop(Location.loc[Location['word'] == 'เกล้า'].index, inplace=True)
Location = Location.reset_index(drop=True)
Location

In [ ]:
Location.drop(Location.loc[Location['word'] == 'ซอย'].index, inplace=True)
Location = Location.reset_index(drop=True)

In [ ]:
for i in range(Location.shape[0]):
  if Location['word'][i] == 'แยก' or Location['word'][i] == 'เเยก':
    Location['word'] = Location['word'].replace(Location['word'][i], 'Noword')

Location.drop(Location.loc[Location['word'] == 'NoWord'].index, inplace=True)
Location = Location.reset_index(drop=True)

In [ ]:
for i in range(Location.shape[0]):
  if Location['word'][i] == 'กทม' or Location['word'][i] == 'กทม.' or Location['word'][i] == 'กรุงเทพมหานคร':
     Location['word'] = Location['word'].replace(Location['word'][i], 'กรุงเทพ')

In [ ]:
for i in range(Location.shape[0]):
  if Location['word'][i] == '':
    Location['word'] = Location['word'].replace(Location['word'][i], 'NoWord')

Location.drop(Location.loc[Location['word'] == 'NoWord'].index, inplace=True)
Location = Location.reset_index(drop=True)
Location

#####Generation Word Cloud

In [ ]:
!pip install wordcloud

In [ ]:
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# Example for Word Cloud
regexp = r"[ก-๙a-zA-Z']+"

In [ ]:
TH_0_words = ''
stopwords = set(thai_stopwords())

for word in Location.word:
  word = str(word)
  # tokens = word_tokenize(word)
  tokens = word.split(" ")
  TH_0_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 1040, height = 1040,
                      background_color ='white',
                      font_path = '/content/drive/MyDrive/NLP/data/ttf/LEELAWAD.ttf',
                      regexp = regexp,
                      min_font_size = 1).generate(TH_0_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (6, 6), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

####Word Rank

In [ ]:
nerword = []
for i in range(Location.shape[0]):
    wordDict = WordCloud(regexp=regexp).process_text(Location.word[i])
    nerword.append(wordDict)

ner_wordFreq_df = pd.DataFrame(nerword)

wordCountList = []
cols = ner_wordFreq_df.columns

for col in cols:
    keyword = col
    freq = ner_wordFreq_df[col].sum()
    wordCountList.append([keyword, freq])

ner_wordCount_df = pd.DataFrame(wordCountList, columns=['Word', 'Freq'])
ner_wordCount_df = ner_wordCount_df.sort_values(by='Freq', ascending=False).reset_index(drop=True)
ner_wordCount_df[0:15]

In [ ]:
mpl.font_manager.fontManager.addfont('/content/drive/MyDrive/DSI444/front/thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New')
plt.rcParams['font.size'] = 14

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.barh(ner_wordCount_df['Word'][0:10], ner_wordCount_df['Freq'][0:10])
ax.invert_yaxis()
ax.set_title('10 สถานที่ที่มีผู้ใช้งานแจ้งมากที่สุดใน Traffy Fondue')
ax.set_ylabel('Word')
ax.set_xlabel('Frequency')
plt.show()